## A webscrapper to collect what yeast strain should be used to make any given style of beer.


In [1]:
# Loading in libraries
from gazpacho import get, Soup
import numpy
import pandas as pd
import requests, bs4
import re, os
import time

### Starting with Inland Island Website

In [2]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://inlandislandyeast.com/suggested-beer-styles/new-england-ipa/'
# fetch step
html = get(url)

# extract step
soup = Soup(html)

In [3]:
# use find() to parse a specific HTML
gaz_data = soup.find('div', {'class':'container'})
gaz_data

[<div class="elementor-container elementor-column-gap-default">
   <div class="elementor-column elementor-col-100 elementor-top-column elementor-element elementor-element-418b4e4" data-id="418b4e4" data-element_type="column">
     <div class="elementor-widget-wrap elementor-element-populated">
       <div class="elementor-element elementor-element-7d18498 header-blue-bg elementor-widget-mobile__width-auto elementor-widget elementor-widget-text-editor" data-id="7d18498" data-element_type="widget" data-widget_type="text-editor.default">
         <div class="elementor-widget-container">
           <p>
             Need Yeast Fast?  
             <a href="#elementor-action%3Aaction%3Dpopup%3Aopen%26settings%3DeyJpZCI6IjUwMDEiLCJ0b2dnbGUiOmZhbHNlfQ%3D%3D">See strains that are nearly always in stock for next-day delivery</a>
           </p>
         </div>
       </div>
     </div>
   </div>
 </div>,
 <div class="elementor-container elementor-column-gap-default">
 					<div class="elementor-

## Pull the Beer Styles into a List

In [4]:
# Taking all the urls and making them a list
ii_url = pd.read_csv('data/iibs.csv')
ii_url = ii_url['Address'].tolist()
ii_url

['https://inlandislandyeast.com/suggested-beer-styles/altbier/',
 'https://inlandislandyeast.com/suggested-beer-styles/amber-lager/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-amber-ale/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-barleywine/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-brown-ale/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-ipa/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-lager/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-light-lager/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-stout/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-wheat-ale/',
 'https://inlandislandyeast.com/suggested-beer-styles/american-wild-ale/',
 'https://inlandislandyeast.com/suggested-beer-styles/australian-sparkling-ale/',
 'https://inlandislandyeast.com/suggested-beer-styles/baltic-porter/',
 'https://inlandislandyeast.com/suggested

In [5]:
# Creating a list of styles from the Inland Island Website
ii_list = pd.read_csv('data/inlandislandbeerstyleslist.csv')
ii_list = ii_list.rename(columns={'Address' : 'Beer_Style'})
ii_list['II_Strains'] = ""
ii_list

,Beer_Style,II_Strains
0,altbier,
1,amber lager,
2,american amber ale,
3,american barleywine,
4,american brown ale,
...,...,...
103,weizenbock,
104,wheatwine,
105,wild specialty beer,
106,witbier,


In [6]:
# Building a definition to pull all the strains suggested for the beer style
# off of each page
def pull_style(soup):
    strain_name = soup.find('div', {'class':'elementor-grid'})
    x = str(strain_name)
    code = 'INIS-'
    res = [i for i in range(len(x)) if x.startswith(code, i )]
    # print(res)
    strains = []
    for i in res:
        strain = x[i:i+8]
        strains.append(strain)
        # print(strain)
    strains = strains[:-10]
    return strains

In [7]:
# Testing out the result
pull_style(soup)

['INIS-445',
 'INIS-446',
 'INIS-318',
 'INIS-003',
 'INIS-008',
 'INIS-009',
 'INIS-960']

In [8]:
# Combining it into a loop

# Setting value for the row index
j = 0

for i in ii_url:
    # fetch step
    html = get(i)
    
    # extract step
    soup = Soup(html)
    
    # print(i)
    x = pull_style(soup)
    print(x)
    # Add pulled list to row
    ii_list.at[j, 'II_Strains'] = x
    
    j = j+1
    
    print(i)

['INIS-572', 'INIS-573', 'INIS-574', 'INIS-576', 'INIS-571', 'INIS-055']
https://inlandislandyeast.com/suggested-beer-styles/altbier/
['INIS-755', 'INIS-761', 'INIS-762', 'INIS-760', 'INIS-763', 'INIS-770', 'INIS-780', 'INIS-702', 'INIS-711']
https://inlandislandyeast.com/suggested-beer-styles/amber-lager/
['INIS-447', 'INIS-442', 'INIS-443', 'INIS-444', 'INIS-445', 'INIS-446', 'INIS-441', 'INIS-301', 'INIS-010']
https://inlandislandyeast.com/suggested-beer-styles/american-amber-ale/
['INIS-317', 'INIS-010', 'INIS-001', 'INIS-004', 'INIS-005', 'INIS-007']
https://inlandislandyeast.com/suggested-beer-styles/american-barleywine/
['INIS-447', 'INIS-446', 'INIS-443', 'INIS-444', 'INIS-445', 'INIS-441', 'INIS-010', 'INIS-001', 'INIS-004']
https://inlandislandyeast.com/suggested-beer-styles/american-brown-ale/
['INIS-012']
https://inlandislandyeast.com/suggested-beer-styles/american-ipa/
['INIS-763', 'INIS-760', 'INIS-770', 'INIS-780', 'INIS-761', 'INIS-762', 'INIS-702', 'INIS-718', 'INIS-71

['INIS-780', 'INIS-761', 'INIS-762', 'INIS-763', 'INIS-760', 'INIS-770', 'INIS-715', 'INIS-702', 'INIS-711']
https://inlandislandyeast.com/suggested-beer-styles/helles/
['INIS-780', 'INIS-761', 'INIS-762', 'INIS-763', 'INIS-721', 'INIS-752', 'INIS-715', 'INIS-718', 'INIS-745']
https://inlandislandyeast.com/suggested-beer-styles/helles-bock/
['INIS-755']
https://inlandislandyeast.com/suggested-beer-styles/helles-lager/
['INIS-801']
https://inlandislandyeast.com/suggested-beer-styles/high-alcohol-beers/
['INIS-447', 'INIS-461', 'INIS-443', 'INIS-444', 'INIS-445', 'INIS-446', 'INIS-442', 'INIS-441', 'INIS-010']
https://inlandislandyeast.com/suggested-beer-styles/ipa/
['INIS-316', 'INIS-301']
https://inlandislandyeast.com/suggested-beer-styles/irish-red-ale/
['INIS-301']
https://inlandislandyeast.com/suggested-beer-styles/irish-stout/
['INIS-970', 'INIS-995', 'INIS-991', 'INIS-992', 'INIS-993', 'INIS-910', 'INIS-446']
https://inlandislandyeast.com/suggested-beer-styles/kettle-sour/
['INIS-

In [9]:
ii_list

,Beer_Style,II_Strains
0,altbier,"[INIS-572, INIS-573, INIS-574, INIS-576, INIS-..."
1,amber lager,"[INIS-755, INIS-761, INIS-762, INIS-760, INIS-..."
2,american amber ale,"[INIS-447, INIS-442, INIS-443, INIS-444, INIS-..."
3,american barleywine,"[INIS-317, INIS-010, INIS-001, INIS-004, INIS-..."
4,american brown ale,"[INIS-447, INIS-446, INIS-443, INIS-444, INIS-..."
...,...,...
103,weizenbock,"[INIS-546, INIS-542, INIS-544, INIS-545, INIS-..."
104,wheatwine,[INIS-055]
105,wild specialty beer,"[INIS-970, INIS-914, INIS-920, INIS-930, INIS-..."
106,witbier,"[INIS-448, INIS-442, INIS-293, INIS-242, INIS-..."


## Propagate Website

Now that I have made it work with one of the websites I will repeat it with the others, next up.... Propagate Lab

I will start again by putting the main url in screaming frog and just pulling all the information that I can after crawling the website to get an index of all the urls.
![title](data/screamingFrog2.png)

In [10]:
# Taking all the urls and making them a list
prop_url = pd.read_csv('data/propagate_url.csv')
prop_url = prop_url['Address'].tolist()
prop_url

['https://propagatelab.com/product-tag/altbier/',
 'https://propagatelab.com/product-tag/american/',
 'https://propagatelab.com/product-tag/american-amber-ale/',
 'https://propagatelab.com/product-tag/american-barleywine/',
 'https://propagatelab.com/product-tag/american-brown-ale/',
 'https://propagatelab.com/product-tag/american-ipa/',
 'https://propagatelab.com/product-tag/american-lager/',
 'https://propagatelab.com/product-tag/american-light-lager/',
 'https://propagatelab.com/product-tag/american-pale-ale/',
 'https://propagatelab.com/product-tag/american-porter/',
 'https://propagatelab.com/product-tag/american-sour-ale/',
 'https://propagatelab.com/product-tag/american-stout/',
 'https://propagatelab.com/product-tag/american-strong-ale/',
 'https://propagatelab.com/product-tag/american-wheat-beer/',
 'https://propagatelab.com/product-tag/australian-sparkling-ale/',
 'https://propagatelab.com/product-tag/baltic-porter/',
 'https://propagatelab.com/product-tag/belgian-blond-ale/'

In [11]:
# Creating a list of styles from the Propagate Website
prop_list = pd.read_csv('data/propagate_styles.csv')
prop_list['Prop_Strains'] = ""
prop_list

,Beer_Style,Prop_Strains
0,altbier,
1,american,
2,american amber ale,
3,american barleywine,
4,american brown ale,
...,...,...
86,wee heavy,
87,weissbier,
88,weizenbock,
89,wheatwine,


In [12]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://propagatelab.com/product-tag/hazy-ipa/'
# fetch step
html = get(url)

# extract step
soup = Soup(html)

In [13]:
print(soup)

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="og: https://ogp.me/ns#">

<head>
	
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1">
	<link rel="profile" href="http://gmpg.org/xfn/11">
		
		<!-- All in One SEO 4.5.8 - aioseo.com -->
		<meta name="robots" content="max-image-preview:large" />
		<link rel="canonical" href="https://propagatelab.com/product-tag/hazy-ipa/" />
		<meta name="generator" content="All in One SEO (AIOSEO) 4.5.8" />
		<script type="application/ld+json" class="aioseo-schema">
			{"@context":"https:\/\/schema.org","@graph":[{"@type":"BreadcrumbList","@id":"https:\/\/propagatelab.com\/product-tag\/hazy-ipa\/#breadcrumblist","itemListElement":[{"@type":"ListItem","@id":"https:\/\/propagatelab.com\/#listItem","position":1,"name":"Home","item":"https:\/\/propagatelab.com\/","nextItem":"https:\/\/propagatelab.com\/product-tag\/hazy-ipa\/#listItem"},{"@type":"ListItem","@id":"https:\/\/propagatelab.com\/product

In [14]:
# Building a definition to pull all the strains suggested for the beer style
# off of each page
def pull_style(soup):
    strain_name = soup.find('span', {'class':'custom_sku'})
    x = str(strain_name)
    # print(x)
    code = 'MIP-'
    res = [i for i in range(len(x)) if x.startswith(code, i )]
    # print(res)
    strains = []
    for i in res:
      strain = x[i:i+7]
      strains.append(strain)
      # print(strain)
    return strains
pull_style(soup)

['MIP-110',
 'MIP-166',
 'MIP-120',
 'MIP-353',
 'MIP-340',
 'MIP-343',
 'MIP-342',
 'MIP-352',
 'MIP-160',
 'MIP-344',
 'MIP-348']

In [15]:
# Combining it into a loop

# Setting value for the row index
j = 0

for i in prop_url:
    # fetch step
    html = get(i)
    
    # extract step
    soup = Soup(html)
    
    # print(i)
    x = pull_style(soup)
    print(x)
    # Add pulled list to row
    prop_list.at[j, 'Prop_Strains'] = x
    
    j = j+1
    
    print(i)

['MIP-511', 'MIP-513', 'MIP-515']
https://propagatelab.com/product-tag/altbier/
['MIP-100', 'MIP-070']
https://propagatelab.com/product-tag/american/
['MIP-100', 'MIP-120', 'MIP-130', 'MIP-170']
https://propagatelab.com/product-tag/american-amber-ale/
['MIP-100', 'MIP-102', 'MIP-130', 'MIP-170']
https://propagatelab.com/product-tag/american-barleywine/
['MIP-100', 'MIP-120', 'MIP-130', 'MIP-170']
https://propagatelab.com/product-tag/american-brown-ale/
['MIP-120', 'MIP-353', 'MIP-340', 'MIP-343', 'MIP-130', 'MIP-352', 'MIP-070', 'MIP-170']
https://propagatelab.com/product-tag/american-ipa/
['MIP-650', 'MIP-610']
https://propagatelab.com/product-tag/american-lager/
['MIP-650', 'MIP-610', 'MIP-070']
https://propagatelab.com/product-tag/american-light-lager/
['MIP-100', 'MIP-120', 'MIP-170']
https://propagatelab.com/product-tag/american-pale-ale/
['MIP-100', 'MIP-102', 'MIP-130', 'MIP-170']
https://propagatelab.com/product-tag/american-porter/
[]
https://propagatelab.com/product-tag/ameri

['MIP-620', 'MIP-000', 'MIP-521', 'MIP-524', 'MIP-600']
https://propagatelab.com/product-tag/weizenbock/
['MIP-100', 'MIP-102', 'MIP-130', 'MIP-170']
https://propagatelab.com/product-tag/wheatwine/
['MIP-232', 'MIP-231', 'MIP-233']
https://propagatelab.com/product-tag/witbier/


In [16]:
prop_list

,Beer_Style,Prop_Strains
0,altbier,"[MIP-511, MIP-513, MIP-515]"
1,american,"[MIP-100, MIP-070]"
2,american amber ale,"[MIP-100, MIP-120, MIP-130, MIP-170]"
3,american barleywine,"[MIP-100, MIP-102, MIP-130, MIP-170]"
4,american brown ale,"[MIP-100, MIP-120, MIP-130, MIP-170]"
...,...,...
86,wee heavy,"[MIP-431, MIP-420]"
87,weissbier,"[MIP-521, MIP-524, MIP-525, MIP-526]"
88,weizenbock,"[MIP-620, MIP-000, MIP-521, MIP-524, MIP-600]"
89,wheatwine,"[MIP-100, MIP-102, MIP-130, MIP-170]"


Now a quick test to make sure I can combine the tables together based on the beer styles, which may not be the same for each of the two companies

In [17]:
# Trying a concat to combine the frames does not work

# Going to try a merge

combined = pd.merge(ii_list, prop_list, how = "outer", on = "Beer_Style")

combined

,Beer_Style,II_Strains,Prop_Strains
0,altbier,"[INIS-572, INIS-573, INIS-574, INIS-576, INIS-...","[MIP-511, MIP-513, MIP-515]"
1,amber lager,"[INIS-755, INIS-761, INIS-762, INIS-760, INIS-...",NaN
2,american amber ale,"[INIS-447, INIS-442, INIS-443, INIS-444, INIS-...","[MIP-100, MIP-120, MIP-130, MIP-170]"
3,american barleywine,"[INIS-317, INIS-010, INIS-001, INIS-004, INIS-...","[MIP-100, MIP-102, MIP-130, MIP-170]"
4,american brown ale,"[INIS-447, INIS-446, INIS-443, INIS-444, INIS-...","[MIP-100, MIP-120, MIP-130, MIP-170]"
...,...,...,...
146,oud bruinpage2,NaN,"[MIP-710, MIP-750]"
147,scottish heavy,NaN,"[MIP-431, MIP-420, MIP-410]"
148,scottish light,NaN,"[MIP-431, MIP-420, MIP-410]"
149,sweet stout,NaN,"[MIP-100, MIP-431, MIP-410, MIP-102, MIP-130]"


In [18]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Beer_Style    151 non-null    object
 1   II_Strains    108 non-null    object
 2   Prop_Strains  91 non-null     object
dtypes: object(3)
memory usage: 3.7+ KB


It looks like the style list will need to be cleaned up a little, I will first just make a global style list from all the suppliers, and then work to clean it up. At the same time I will pull all the supplier strains for the styles as well.

## White Labs Website

In [19]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://www.whitelabs.com/yeast-single?id=103&type=YEAST'
# fetch step
html = get(url)

# extract step
soup = Soup(html)

In [20]:
# use find() to parse a specific HTML
gaz_data = soup.find('ul', {'class':'beer_style'})
gaz_data

<ul class="beer_style">
  <li>
Altbier
</li>
  <li>
Cream Ale
</li>
  <li>
Kolsch
</li>
</ul>

In [36]:
# Building a definition to pull all the strains suggested for the beer style
# off of each page
def pull_strain(soup):
    
    # pulling the strain name
    strain_name = soup.find('ul', {'class' : 'details'})
    # print(strain_name)
    strain_name = str(strain_name)
    s = strain_name.find('WLP')
    strain_name = strain_name[s:s+6]
    print(strain_name)
    return strain_name
    
    # Pulling the Beer Styles

def pull_style(soup):
    style_name = soup.find('ul', {'class':'beer_style'})
    x = str(style_name)
    code = '<li>'
    res = [i for i in range(len(x)) if x.startswith(code, i )]
    # print(res)
    styles = []
    for i in res:
        style = x[i+4:i+24]
        style = style.replace('\n', '')
        style = style.replace('</li>', '')
        style = style.replace('<li>', '')
        style = style.replace('</li', '')
        style = style.replace('<li', '')
        style = style.replace('<l', '')
        style = style.replace('</', '')
        style = style.replace('<', '')
        styles.append(style)
        # print(styles)
    return styles


In [37]:
pull_strain(soup)

WLP003


'WLP003'

In [38]:
pull_style(soup)

['Altbier  ', 'Cream Ale  ', 'Kolschul>']

In [22]:
# Upload the White Labs URL List
# Taking all the urls and making them a list
wlp_url = pd.read_csv('data/whitelabs_url.csv')
wlp_url = wlp_url['Address'].tolist()
wlp_url

['https://www.whitelabs.com/yeast-single?id=101&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=102&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=103&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=104&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=105&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=106&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=107&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=108&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=109&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=110&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=111&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=112&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=113&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=114&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=115&type=YEAST',
 'https://www.whitelabs.com/yeast-single?id=116&type=YEAST',
 'https://www.whitelabs.

### Creating a list of all the strains


In [46]:
# Combining it into a loop

# create a blank dataframe

columns = {'Strain':[], 'Style':[]}
wlp_list = pd.DataFrame(data=columns)

print(wlp_list)

# Setting value for the row index
for i in wlp_url:
    # fetch step
    html = get(i)
    
    # extract step
    soup = Soup(html)
    
    # print(i)
    x = pull_strain(soup)
    y = pull_style(soup)
    # print(x)
    # print(y)
    # Add pulled list to row
    new_row = {'Strain' : x, 'Style' : y}
    print(new_row)
    wlp_list = pd.concat([wlp_list, pd.DataFrame([new_row])], ignore_index = True)
    

Empty DataFrame
Columns: [Strain, Style]
Index: []
WLP001
{'Strain': 'WLP001', 'Style': ['American IPA', 'American Wheat Beer', 'Barleywine  ', 'Blonde Ale  ', 'Brown Ale  ', 'California Common', 'Cider  ', 'Double IPA  ', 'Dry Mead  ', 'Imperial Stout', 'Old Ale  ', 'Pale Ale  ', 'Porter  ', 'Red Ale  ', 'Stout  ', 'Sweet Mead']}
WLP002
{'Strain': 'WLP002', 'Style': ['American IPA', 'Blonde Ale  ', 'Brown Ale  ', 'Double IPA  ', 'English Bitter', 'English IPA ', 'Hazy/Juicy IPA', 'Imperial Stout', 'Old Ale  ', 'Pale Ale  ', 'Porter  ', 'Red Ale  ', 'Stoutul>']}
WLP003
{'Strain': 'WLP003', 'Style': ['Altbier  ', 'Cream Ale  ', 'Kolschul>']}
WLP004
{'Strain': 'WLP004', 'Style': ['Blonde Ale  ', 'Brown Ale  ', 'Cider  ', 'Dry Mead  ', 'English Bitter', 'English IPA ', 'Porter  ', 'Red Ale  ', 'Scotch Ale  ', 'Stout  ', 'Sweet Mead']}
WLP005
{'Strain': 'WLP005', 'Style': ['Barleywine  ', 'Blonde Ale  ', 'Brown Ale  ', 'Cider  ', 'Dry Mead  ', 'English Bitter', 'English IPA ', 'Imperial St

WLP300
{'Strain': 'WLP300', 'Style': ['Weissbier  ', 'Weizenbock']}
WLP320
{'Strain': 'WLP320', 'Style': ['American Wheat Beer']}

{'Strain': '', 'Style': ['Additional Beverage', 'Additional Styles']}
WLP351
{'Strain': 'WLP351', 'Style': ['Weissbier  ', 'Weizenbock']}
WLP380
{'Strain': 'WLP380', 'Style': ['Weissbier  ', 'Weizenbock']}
WLP400
{'Strain': 'WLP400', 'Style': ['Belgian Pale Ale', 'Belgian Tripel', 'Cider  ', 'Saison  ', 'Witbierul>']}
WLP410
{'Strain': 'WLP410', 'Style': ['Belgian Dubbel', 'Belgian Pale Ale', 'Belgian Tripel', 'Cider  ', 'Saison  ', 'Witbierul>']}
WLP500
{'Strain': 'WLP500', 'Style': ['Belgian Dark Strong', 'Belgian Dubbel', 'Belgian Pale Ale', 'Belgian Tripel', 'Biere de Garde', 'Ciderul>']}
WLP510
{'Strain': 'WLP510', 'Style': ['Belgian Dark Strong', 'Belgian Dubbel', 'Belgian Pale Ale', 'Belgian Tripel', 'Ciderul>']}
WLP515
{'Strain': 'WLP515', 'Style': ['Belgian Dubbel', 'Belgian Pale Ale', 'Belgian Tripel']}
WLP518
{'Strain': 'WLP518', 'Style': ['Ameri

WLP833
{'Strain': 'WLP833', 'Style': ['Amber Lager ', 'Baltic Porter', 'Bock  D', 'Dark Lager  ', 'Dopplebock  ', 'Helles  ', 'Pilsnerul>']}
WLP835
{'Strain': 'WLP835', 'Style': ['Amber Lager ', 'Bock  D', 'Dark Lager  ', 'Dopplebock  ', 'Helles  ', 'Marzen  ', 'Pale Lager  ', 'Pilsner  ', 'Rauchbier  ', 'Schwarzbier ', 'Vienna Lager']}
WLP838
{'Strain': 'WLP838', 'Style': ['Amber Lager ', 'American Lager', 'Bock  C', 'Cream Ale  ', 'Dark Lager  ', 'Dopplebock  ', 'Helles  ', 'Marzen  ', 'Pale Lager  ', 'Pilsner  ', 'Schwarzbier ', 'Vienna Lager']}
WLP840
{'Strain': 'WLP840', 'Style': ['Amber Lager ', 'American Lager', 'Dark Lager  ', 'Helles  ', 'Pale Lager']}
WLP845
{'Strain': 'WLP845', 'Style': ['Amber Lager ', 'American Lager', 'Cream Ale  ', 'Dark Lager  ', 'Pale Lager']}
WLP850
{'Strain': 'WLP850', 'Style': ['Amber Lager ', 'American Lager', 'Dark Lager  ', 'Pale Lager  ', 'Schwarzbier ', 'Vienna Lager']}
WLP860
{'Strain': 'WLP860', 'Style': ['Amber Lager ', 'American Lager', 'Bo

In [47]:
wlp_list

,Strain,Style
0,WLP001,"[American IPA, American Wheat Beer, Barleywine..."
1,WLP002,"[American IPA, Blonde Ale , Brown Ale , Doub..."
2,WLP003,"[Altbier , Cream Ale , Kolschul>]"
3,WLP004,"[Blonde Ale , Brown Ale , Cider , Dry Mead ..."
4,WLP005,"[Barleywine , Blonde Ale , Brown Ale , Cide..."
...,...,...
190,WLP400,[]
191,WLP096,[]
192,WLP808,"[Additional Styles, Amber Lager , American Lag..."
193,WLP468,"[Additional Styles, Flanders Red Ale, Lambicul>]"
